In [1]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    hosts="https://ae4da19a26f0488b87b8f5f2bb35f565.eastus2.azure.elastic-cloud.com",
    basic_auth=("elastic", "6AnfGz6hxlSwceSaE5xSsVsd")
)

es_client.ping()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key="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",
)

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

doc_loader = PyPDFLoader("../doc/First Edition English 2023.pdf")
document = doc_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, 
    chunk_overlap=0, 
    separators="\n\n"
)
splitted_documents = text_splitter.split_documents(documents=document)

In [40]:
# def stop_on_second_occurrence(paragraphs, keywords):

index_text = ""
keywords = ["Laws", "Decrees", "Executive Council Resolutions, Chairman of the Executive Council Resolutions"]
occurrence_count = {keyword: 0 for keyword in keywords}
stop_processing = False
val = 0
for chunk in splitted_documents:
    if stop_processing:
        break 
    content = chunk.page_content
    for keyword in keywords:
        if keyword in content:
            occurrence_count[keyword] += 1
            if occurrence_count[keyword] == 2:
                stop_processing = True
                break
    index_text = index_text + content
    val = val + 1


In [41]:
print(val)

5


In [43]:
print(index_text)

The Ofﬁcial
Gazette
31 January 2023 - The First Edition1The Official Gazette
The First Edition - The Fifty-Second Year
Contents
Laws
Law No. (16) of 2022 Concerning The Licensing of The Drivers and 
Vehicles in The Emirate of Abu Dhabi.
Law No. (1) of 2023 Concerning the establishment of Abu Dhabi 
Transport Company “Public Joint Stock Company”.
Law No. (4) of 2023 Concerning The Establishment of Abu Dhabi 
Media Office.
Decrees
Amiri Decree No. (3) of 2023 Concerning Some Members of the 
Executive Council in the Emirate of Abu Dhabi.
Executive Council Resolutions
Executive Council Resolution No. (169) of 2022 Concerning the 
Thirty-First Investment Zone in the Emirate of Abu Dhabi.
Executive Council Resolution No. (182) of 2022 Concerning the 
General Policy for the Management, Regulation and Protection of 
Groundwater in the Emirate of Abu Dhabi.
*Attachment: General Policy for the Management, Regulation and Protection of 
Groundwater in the Emirate of Abu Dhabi
Executive Council Res

In [44]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an expert at identifying the Laws, Decrees and Executive Council Resolutions 
    from the given paragraph: {paragraph} 
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["paragraph"],
)

question_router = prompt | llm
paragraph = splitted_documents[1].page_content + '/n' + splitted_documents[2].page_content + '/n' + splitted_documents[3].page_content

response = question_router.invoke({"paragraph": index_text})


In [45]:
print(response.content)

Based on the given text, here are the Laws, Decrees, and Executive Council Resolutions identified:

**Laws:**

1. Law No. (16) of 2022: Concerning The Licensing of The Drivers and Vehicles in The Emirate of Abu Dhabi.
2. Law No. (1) of 2023: Concerning the establishment of Abu Dhabi Transport Company “Public Joint Stock Company”.
3. Law No. (4) of 2023: Concerning The Establishment of Abu Dhabi Media Office.

**Decrees:**

1. Amiri Decree No. (3) of 2023: Concerning Some Members of the Executive Council in the Emirate of Abu Dhabi.

**Executive Council Resolutions:**

1. Executive Council Resolution No. (169) of 2022: Concerning the Thirty-First Investment Zone in the Emirate of Abu Dhabi.
2. Executive Council Resolution No. (182) of 2022: Concerning the General Policy for the Management, Regulation and Protection of Groundwater in the Emirate of Abu Dhabi.
3. Executive Council Resolution No. (183) of 2022: Concerning the Amendment of Some Provisions of Chairman of the Executive Counci

In [46]:
pip install PyPDF2


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [47]:
from PyPDF2 import PdfReader

def find_empty_pages(pdf_path):
    empty_pages = []
    
    # Read the PDF file
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)
        
        for i in range(num_pages):
            page = reader.pages[i]
            text = page.extract_text()  # Extract text from the page
            
            # Check if the text is None or empty
            if text is None or text.strip() == "":
                empty_pages.append(i + 1)  # Store the page number (1-indexed)

    return empty_pages

# Example usage
pdf_path = "../doc/First Edition English 2023.pdf"
empty_pages = find_empty_pages(pdf_path)
print("Empty pages:", empty_pages)


Empty pages: [5, 23, 25, 29, 66]


In [68]:
pdf_path = "../doc/First Edition English 2023.pdf"
with open(pdf_path, 'rb') as file:
    reader = PdfReader(file)
    num_pages = len(reader.pages)
    page = reader.pages[31]
    text = page.extract_text()
    print(text)

31



In [63]:
import PyPDF2

def find_sentence_in_pdf(pdf_path, sentence):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_number in range(3, len(reader.pages)):
            page = reader.pages[page_number]
            text = page.extract_text()
            if text and sentence in text:
                print(f'Sentence found on page {page_number}')

# Example usage
pdf_path = '../doc/First Edition English 2023.pdf'
sentence_to_find = 'Executive Council Resolutions'
find_sentence_in_pdf(pdf_path, sentence_to_find)


Sentence found on page 27


In [ ]:
pdf_path = "../doc/First Edition English 2023.pdf"
with open(pdf_path, 'rb') as file:
    reader = PdfReader(file)
    num_pages = len(reader.pages)
    page = reader.pages[31]
    text = page.extract_text()
    print(text)